In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.distributed as dist

import time
import os
import sys
from BOW_model_b import BOW_model

In [3]:
glove_embeddings = np.load('/Users/liuchunlei/Desktop/IMDB Movie reviews/preprocessed_data/glove_embeddings.npy')
vocab_size = 8000

x_train = []
with open('/Users/liuchunlei/Desktop/IMDB Movie reviews/preprocessed_data/imdb_train_glove.txt','r',encoding='utf-8') as f:
    lines = f.readlines()
for line in lines:
    line = line.strip()
    line = line.split(' ')
    line = np.asarray(line,dtype=np.int)

    line[line>vocab_size] = 0

    line = np.mean(glove_embeddings[line],axis=0) #vertically
    x_train.append(line)
x_train = np.asarray(x_train)
x_train = x_train[0:25000]
y_train = np.zeros((25000,))
y_train[0:12500] = 1

x_test = []
with open('/Users/liuchunlei/Desktop/IMDB Movie reviews/preprocessed_data/imdb_test_glove.txt','r',encoding='utf-8') as f:
    lines = f.readlines()
for line in lines:
    line = line.strip()
    line = line.split(' ')
    line = np.asarray(line,dtype=np.int)

    line[line>vocab_size] = 0
    
    line = np.mean(glove_embeddings[line],axis=0)

    x_test.append(line)
x_test = np.asarray(x_test)
y_test = np.zeros((25000,))
y_test[0:12500] = 1

vocab_size += 1
batch_size = 200
no_of_epochs = 20
model = BOW_model(no_of_hidden_units = 300)
# model.cuda()
# opt = 'sgd'
# LR = 0.01
opt = 'adam'
LR = 0.001
if(opt=='adam'):
    optimizer = optim.Adam(model.parameters(), lr=LR)
elif(opt=='sgd'):
    optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9)
L_Y_train = len(y_train) #25000
L_Y_test = len(y_test)


train_loss = []
train_accu = []
test_accu = []



In [42]:
for epoch in range(no_of_epochs):

    # training
    model.train()

    epoch_acc = 0.0
    epoch_loss = 0.0

    epoch_counter = 0

    time1 = time.time()
    
    I_permutation = np.random.permutation(L_Y_train)

    for i in range(0, L_Y_train, batch_size):

        x_input = x_train[I_permutation[i:i+batch_size]]
        y_input = y_train[I_permutation[i:i+batch_size]]
#         x_input = Variable(torch.FloatTensor(x_input)).cuda()
#         target = Variable(torch.FloatTensor(y_input)).cuda()
        x_input = torch.FloatTensor(x_input)
        target = torch.FloatTensor(y_input)
        optimizer.zero_grad()
        loss, pred = model(x_input,target)
        loss.backward()

        optimizer.step()   # update weights
        
        prediction = pred >= 0.0
        truth = target >= 0.5
        acc = prediction.eq(truth).sum().cpu().data.numpy()
        epoch_acc += acc
        epoch_loss += loss.data[0]
        epoch_counter += batch_size

    epoch_acc /= epoch_counter
    epoch_loss /= (epoch_counter/batch_size)

    train_loss.append(epoch_loss)
    train_accu.append(epoch_acc)

    print(epoch, "%.2f" % (epoch_acc*100.0), "%.4f" % epoch_loss, "%.4f" % float(time.time()-time1))

    # ## test
    model.eval()

    epoch_acc = 0.0
    epoch_loss = 0.0

    epoch_counter = 0

    time1 = time.time()
    
    I_permutation = np.random.permutation(L_Y_test)

    for i in range(0, L_Y_test, batch_size):

        x_input = x_test[I_permutation[i:i+batch_size]]
        y_input = y_test[I_permutation[i:i+batch_size]]
#         x_input = Variable(torch.FloatTensor(x_input)).cuda()
#         target = Variable(torch.FloatTensor(y_input)).cuda()
        x_input = torch.FloatTensor(x_input)
        target = torch.FloatTensor(y_input)
        loss, pred = model(x_input,target)
        
        prediction = pred >= 0.0
        truth = target >= 0.5
        acc = prediction.eq(truth).sum().cpu().data.numpy()
        epoch_acc += acc
        epoch_loss += loss.data[0]
        epoch_counter += batch_size

    epoch_acc /= epoch_counter
    epoch_loss /= (epoch_counter/batch_size)

    test_accu.append(epoch_acc)

    time2 = time.time()
    time_elapsed = time2 - time1

    print("  ", "%.2f" % (epoch_acc*100.0), "%.4f" % epoch_loss)

torch.save(model,'BOW_b.model')
data = [train_loss,train_accu,test_accu]
data = np.asarray(data)
np.save('data.npy',data)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


0 79.69 0.4432 0.6377


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:70: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


   82.18 0.3999
1 83.03 0.3853 0.6022
   76.42 0.4760
2 83.62 0.3724 0.5833
   79.09 0.4470
3 83.97 0.3661 0.5810
   82.88 0.3834
4 84.20 0.3594 0.5823
   74.96 0.5467
5 84.51 0.3540 0.6264
   83.45 0.3746
6 84.65 0.3499 0.6471
   84.19 0.3610
7 84.92 0.3461 0.5966
   83.70 0.3687
8 84.97 0.3443 0.5974
   83.74 0.3690
9 85.34 0.3397 0.6481
   82.75 0.3892
10 85.38 0.3343 0.6311
   84.27 0.3593
11 85.49 0.3348 0.6127
   83.06 0.3789
12 85.72 0.3281 0.6090
   78.21 0.4674
13 85.75 0.3288 0.5847
   81.00 0.4246
14 86.19 0.3218 0.5814
   81.65 0.4177
15 86.28 0.3216 0.6384
   83.50 0.3760
16 86.68 0.3166 0.5893
   84.01 0.3667
17 86.54 0.3155 0.5785
   82.96 0.3887
18 86.34 0.3162 0.5983
   76.30 0.5651
19 86.48 0.3129 0.6571
   82.44 0.4049


The training accuracy for part 1b stops much earlier (around 86-88%) and doesn’t seem to improve much more. <br>
Nearly 95% of the weights belong to the embedding layer in part 1a. We’re training significantly less in part 1b and can’t actually fine-tune the word embeddings at all. Using only 300 hidden weights for part 1b results in very little overfitting while still achieving decent accuracy.